In [1]:
import os
import math

In [2]:
import pandas as pd
import torch
import transformers
from transformers import AdamW, BertConfig
from sklearn.metrics import precision_recall_fscore_support

In [3]:
import conll04_constants as constants
import conll04_input_generator as input_generator
import model

In [4]:
from tqdm import tqdm

In [5]:
model_save_path = "../../model/spert/conll04_"

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
entity_label_map = {v: k for k, v in input_generator.parser.entity_encode.items()}
entity_classes = list(entity_label_map.keys())
entity_classes.remove(0)

In [8]:
relation_label_map = {v: k for k, v in input_generator.parser.relation_encode.items()}
relation_classes = list(relation_label_map.keys())
relation_classes.remove(0)

In [9]:
def get_optimizer_params(model):
    params = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_params = [
        {"params": [p for n, p in params if not any(nd in n for nd in no_decay)],
         "weight_decay": constants.weight_decay},
        {"params": [p for n, p in params if any(nd in n for nd in no_decay)],
         "weight_decay": 0.0}
    ]
    return optimizer_params

In [10]:
def evaluate_results(true_labels, predicted_labels, label_map, classes):
    """Compare the prediction list with the truth values"""
    precision, recall, fbeta_score, support \
        = precision_recall_fscore_support(true_labels, predicted_labels, average=None, labels=classes, 
                                          zero_division=0)
    result = pd.DataFrame(index=[label_map[c] for c in classes])
    result["precision"] = precision
    result["recall"] = recall
    result["fbeta_score"] = fbeta_score
    result["support"] = support
    result.loc["macro"] = list(precision_recall_fscore_support(true_labels, predicted_labels, average="macro", 
                                                               labels=classes, zero_division=0))
    result.loc["micro"] = list(precision_recall_fscore_support(true_labels, predicted_labels, average="micro", 
                                                               labels=classes, zero_division=0))
    return result

In [11]:
def evaluate(spert_model, group):
    spert_model.eval()
    eval_entity_span_pred = []
    eval_entity_span_true = []
    eval_entity_embedding_pred = []
    eval_entity_embedding_true = []
#     eval_relation
#     eval_relation_pred = []
#     eval_relation_true = []
    eval_generator = input_generator.data_generator(group, device, 
                                                    is_training=False,
                                                    neg_entity_count=constants.neg_entity_count, 
                                                    neg_relation_count=constants.neg_relation_count, 
                                                    max_span_size=constants.max_span_size)
    eval_dataset = list(eval_generator)
    eval_size = len(eval_dataset)
    for inputs, infos in tqdm(eval_dataset, total=eval_size, desc="Evaluation "+group):
        # forward
        outputs = spert_model(**inputs, is_training=False)
        
        # retrieve results for evaluation
        eval_entity_span_pred += outputs["entity"]["pred"].tolist()
        eval_entity_span_true += inputs["entity_label"].tolist()
        eval_entity_embedding_pred += outputs["entity"]["embedding"].tolist()
        eval_entity_embedding_true += infos["entity_embedding"].tolist()
#         eval_relation_pred += [] if outputs["relation"] == None else outputs["relation"]["pred"].tolist()
#         eval_relation_true += inputs["relation_label"].tolist()
        assert len(eval_entity_span_pred) == len(eval_entity_span_true)
        assert len(eval_entity_embedding_pred) == len(eval_entity_embedding_true)
#         assert len(eval_relation_pred) == len(eval_relation_true)
        
    # evaluate & save
    results = pd.concat([
        evaluate_results(eval_entity_span_true, eval_entity_span_pred, entity_label_map, entity_classes),
        evaluate_results(eval_entity_embedding_true, eval_entity_embedding_pred, entity_label_map, 
                         entity_classes)#,
#         evaluate_results(eval_relation_true, eval_relation_pred, relation_label_map, relation_classes)
    ], keys=["Entity span", "Entity embedding"])# , "Relation"])
    results.to_csv(model_save_path + "evaluate_" + group + ".csv")
    print(results)

In [12]:
def train():
    """Train the model and evaluate on the dev dataset
    """
    
    train_group = "train"
    validation_group = "dev"
    
    # Training
    train_generator = input_generator.data_generator(train_group, device, 
                                                     is_training=True,
                                                     neg_entity_count=constants.neg_entity_count, 
                                                     neg_relation_count=constants.neg_relation_count, 
                                                     max_span_size=constants.max_span_size)
    train_dataset = list(train_generator)
    train_size = len(train_dataset)
    
    config = BertConfig.from_pretrained(constants.model_path)
    spert_model = model.SpERT.from_pretrained(constants.model_path,
                                              # SpERT model parameters
                                              relation_types=constants.relation_types, 
                                              entity_types=constants.entity_types, 
                                              width_embedding_size=constants.width_embedding_size, 
                                              prop_drop=constants.prop_drop, 
                                              freeze_transformer=True, 
                                              max_pairs=constants.max_pairs, 
                                              is_overlapping=False, 
                                              relation_filter_threshold=constants.relation_filter_threshold)
    spert_model.train()
    spert_model.to(device)
    optimizer_params = get_optimizer_params(spert_model)
    optimizer = AdamW(optimizer_params, lr=constants.lr, weight_decay=constants.weight_decay, correct_bias=False)
    scheduler = transformers.get_linear_schedule_with_warmup(optimizer, 
                             num_warmup_steps=constants.lr_warmup*train_size*constants.epochs, 
                             num_training_steps=train_size*constants.epochs)
    
    for epoch in range(constants.epochs):
        losses = []
        train_entity_span_pred = []
        train_entity_span_true = []
        train_entity_embedding_pred = []
        train_entity_embedding_true = []
        train_relation_pred = []
        train_relation_true = []
        for inputs, infos in tqdm(train_dataset, total=train_size, desc='Train epoch %s' % epoch):
            spert_model.zero_grad()
            
            # forward
            outputs = spert_model(**inputs, is_training=True)
            
            # backward
            loss = outputs["loss"]
            loss.backward()
            torch.nn.utils.clip_grad_norm_(spert_model.parameters(), constants.max_grad_norm)
            optimizer.step()
            scheduler.step()
            # print(optimizer.param_groups[0]['lr'])
            
            # retrieve results for evaluation
            losses.append(loss.item())
            train_entity_span_pred += outputs["entity"]["pred"].tolist()
            train_entity_span_true += inputs["entity_label"].tolist()
            train_entity_embedding_pred += outputs["entity"]["embedding"].tolist()
            train_entity_embedding_true += infos["entity_embedding"].tolist()
            train_relation_pred += [] if outputs["relation"] == None else outputs["relation"]["pred"].tolist()
            train_relation_true += inputs["relation_label"].tolist()
            assert len(train_entity_span_pred) == len(train_entity_span_true)
            assert len(train_entity_embedding_pred) == len(train_entity_embedding_true)
            assert len(train_relation_pred) == len(train_relation_true)
            
        # evaluate & save checkpoint
        print("epoch:", epoch,"average loss:", sum(losses) / len(losses))
        results = pd.concat([
            evaluate_results(train_entity_span_true, train_entity_span_pred, entity_label_map, entity_classes),
            evaluate_results(train_entity_embedding_true, train_entity_embedding_pred, entity_label_map, 
                             entity_classes),
            evaluate_results(train_relation_true, train_relation_pred, relation_label_map, relation_classes)
        ], keys=["Entity span", "Entity embedding", "Relation"])
        results.to_csv(model_save_path + "epoch_" + str(epoch) + ".csv")
        evaluate(spert_model, validation_group)
    torch.save(spert_model.state_dict(), model_save_path + "epoch_" + str(constants.epochs-1) + ".model")

In [13]:
train()

Some weights of the model checkpoint at bert-base-cased were not used when initializing SpERT: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing SpERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SpERT from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of SpERT were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['relation_classifier.weight', 'relation_classifier.bi

epoch: 0 average loss: 0.7844143317623453


Train epoch 1:   0%|▎                                                                  | 4/910 [00:00<00:26, 34.48it/s]

                          precision  recall  fbeta_score  support
Entity span      I-Loc          0.0     0.0          0.0    337.0
                 I-Peop         0.0     0.0          0.0    225.0
                 I-Org          0.0     0.0          0.0    190.0
                 I-Other        0.0     0.0          0.0    125.0
                 macro          0.0     0.0          0.0      NaN
                 micro          0.0     0.0          0.0      NaN
Entity embedding I-Loc          0.0     0.0          0.0    717.0
                 I-Peop         0.0     0.0          0.0    793.0
                 I-Org          0.0     0.0          0.0    747.0
                 I-Other        0.0     0.0          0.0    354.0
                 macro          0.0     0.0          0.0      NaN
                 micro          0.0     0.0          0.0      NaN


Train epoch 1: 100%|█████████████████████████████████████████████████████████████████| 910/910 [00:26<00:00, 34.39it/s]


...

                          precision    recall  fbeta_score  support
Entity span      I-Loc     0.584112  0.370920     0.453721    337.0
                 I-Peop    0.449153  0.235556     0.309038    225.0
                 I-Org     0.338710  0.110526     0.166667    190.0
                 I-Other   0.214286  0.048000     0.078431    125.0
                 macro     0.396565  0.191250     0.251964      NaN
                 micro     0.485782  0.233751     0.315627      NaN
Entity embedding I-Loc     0.890995  0.262204     0.405172    717.0
                 I-Peop    0.993631  0.196721     0.328421    793.0
                 I-Org     0.842105  0.064257     0.119403    747.0
                 I-Other   0.913043  0.059322     0.111406    354.0
                 macro     0.909944  0.145626     0.241101      NaN
                 micro     0.921875  0.158177     0.270023      NaN


Train epoch 19: 100%|████████████████████████████████████████████████████████████████| 910/910 [00:27<00:00, 33.29it/s]


epoch: 19 average loss: 0.14776028453648746


Evaluation dev: 100%|████████████████████████████████████████████████████████████████| 243/243 [00:09<00:00, 25.93it/s]


                          precision    recall  fbeta_score  support
Entity span      I-Loc     0.588785  0.373887     0.457350    337.0
                 I-Peop    0.443478  0.226667     0.300000    225.0
                 I-Org     0.338710  0.110526     0.166667    190.0
                 I-Other   0.214286  0.048000     0.078431    125.0
                 macro     0.396315  0.189770     0.250612      NaN
                 micro     0.486874  0.232611     0.314815      NaN
Entity embedding I-Loc     0.891509  0.263598     0.406889    717.0
                 I-Peop    0.993506  0.192938     0.323126    793.0
                 I-Org     0.842105  0.064257     0.119403    747.0
                 I-Other   0.913043  0.059322     0.111406    354.0
                 macro     0.910041  0.145029     0.240206      NaN
                 micro     0.921525  0.157411     0.268891      NaN


In [16]:
config = BertConfig.from_pretrained(constants.model_path)
spert_model = model.SpERT.from_pretrained(constants.model_path,
                                          # SpERT model parameters
                                          relation_types=constants.relation_types, 
                                          entity_types=constants.entity_types, 
                                          width_embedding_size=constants.width_embedding_size, 
                                          prop_drop=constants.prop_drop, 
                                          freeze_transformer=True, 
                                          max_pairs=constants.max_pairs, 
                                          is_overlapping=False, 
                                          relation_filter_threshold=constants.relation_filter_threshold)
spert_model.to(device)
# state_dict = torch.load(model_save_path + "epoch_" + str(constants.epochs-1) + ".model")
state_dict = torch.load("../../model/spert/pytorch_model.bin")
state_dict["relation_classifier.weight"] = state_dict["rel_classifier.weight"]
state_dict["relation_classifier.bias"] = state_dict["rel_classifier.bias"]
state_dict["width_embedding.weight"] = state_dict["size_embeddings.weight"]
spert_model.load_state_dict(state_dict, strict=False)

Some weights of the model checkpoint at bert-base-cased were not used when initializing SpERT: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing SpERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SpERT from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of SpERT were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['relation_classifier.weight', 'relation_classifier.bi

_IncompatibleKeys(missing_keys=['bert.embeddings.position_ids'], unexpected_keys=['rel_classifier.weight', 'rel_classifier.bias', 'size_embeddings.weight'])

In [17]:
evaluate(spert_model, "test")

Evaluation test: 100%|███████████████████████████████████████████████████████████████| 288/288 [00:12<00:00, 23.24it/s]


                          precision    recall  fbeta_score  support
Entity span      I-Loc     0.422505  0.932084     0.581446    427.0
                 I-Peop    0.000000  0.000000     0.000000    321.0
                 I-Org     0.001401  0.005051     0.002193    198.0
                 I-Other   0.531579  0.759398     0.625387    133.0
                 macro     0.238871  0.424133     0.302257      NaN
                 micro     0.226552  0.463392     0.304321      NaN
Entity embedding I-Loc     0.946203  0.929534     0.937794    965.0
                 I-Peop    0.004144  0.002671     0.003249   1123.0
                 I-Org     0.002655  0.004172     0.003245    719.0
                 I-Other   0.819767  0.834320     0.826979    338.0
                 macro     0.443192  0.442674     0.442817      NaN
                 micro     0.376669  0.376789     0.376729      NaN
